#Ejercicio 1: Crea un prompt temático

El enunciado de este primer ejercicio es el siguiente: En este ejercicio, deberás diseñar un prompt utilizando las técnicas explicadas en clase, basado en una temática específica de tu elección. El objetivo es que el prompt sea claro, coherente y detallado, permitiendo obtener una respuesta precisa y
relevante.

Como puede apreciarse, lo que se solicita en este ejercicio es generar un prompt automático sobre una temática a elegir y que en base este prompt, se obtenga una respuesta acorde a lo preguntado. Para ello, lo primero que hay que hacer es decidir la temática. Dado que se debe solicitar al usuario una serie de campos y en base a esa información se tiene que generar el prompt, la temática a elegir debe ser una que permita solicitar varios apartados al usuario.

Tras haberlo pensado he decidido que la temática del prompt va a ser la cocina y concretamente la generación de recetas ya que entre recetas hay muchos campos que se repiten (tiempo de ejecución, nivel de dificultad, ingredientes...) lo caul pueden ser los campos que se le soliciten al usuario. Es por eso que debido a su gran versatilidad, decido decantarme por esta temática.

El modo en que voy a enfocar el ejercicio es el siguiente. En primer lugar, se solicitará al usuario que complete los campos específicos para la receta. Con la información de estos campos generaré el prompt y este prompt lo introduciré por medio de la API de OpenAI como un prompt de usuario. Finalmente, capturaré el mensaje obtenido y se imprimirá por pantalla la receta.

Por ello, dado que se va utilizar openai y su API, el primer paso es instalar la librería.

In [10]:
!pip install openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00


A continuación, voy a generar una función que es la que se va a encargar de generar el prompt. Voy a trabajar con funciones para que el ejercicio este más organizado y más limpio. Esta primera función se va a denominar generar_prompt y en ella se va a solicitar al usuario que complete una serie de campos y como respuesta se va a generar el prompt. Los campos que el usuario va a tener que completar son:

* Tipo de cocina: Como puede ser una receta genérica, el usuario va a tener que especificar el tipo de cocina que desea cocinar. Por ejemplo, mexicana, italiana, japonesa...
* Tiempo de preparación: El tiempo que el usuario desea emplear cocinando.
* Número de porciones: Es necesario conocer el número de comensales para poder ajustar la receta.
* Nivel de dificultad: No es lo mismo cocinar sushi que cocinar gyozas. Aunque ambas son cocina japonesa, el grado de dificultad no es el mismo.
* Ingrediente principal: En la mayoría de recetas se suele indicar cual es el ingrediente principal, por lo que dejo la puerta abierta a esta posibilidad.
* Equipamiento disponible: Puede ser interesante ofrecer la posibilidad de especificar el equipamiento del que se dispone para cocinar.
* Preferencia dietética: Hoy en día existe mucha conciencia con la preferencia dietética por lo que me parece importante incluir este campo en la receta. Con preferencia dietética me refiero a opciones como vegano, vegetariano, sin gluten...

Los campos que he definido no son igual de importantes desde mi punto de vista dado que para generar la recet no es necesario conocer la información de todos ellos. Por ejemplo, el campo de equipamiento disponible no es igual de importante que el campo tipo de cocina. Es por eso que los últimos tres campos van a ser opcionales para que el usuario decida si quiere completarlos o no. En caso de que se complete, al prompt se le añadirá la información adicional.  

In [ ]:
import openai
from openai import OpenAI

def generar_prompt():
    #CAMPOS REQUERIDOS
    tipo_cocina = input("Ingrese el tipo de cocina que desea cocinar: ")
    tiempo_preparacion = input("Ingrese el tiempo de preparación (en minutos): ")
    num_porciones = input("Ingrese el número de porciones: ")
    nivel_dificultad = input("Ingrese el nivel de dificultad: fácil, intermedio o avanzado): ")

    #CAMPOS ADICIONALES
    ingredientes_principales = input("¿Cuáles son los ingredientes principales que desea usar? [Opcional]: ")
    equipamiento_disponible = input("Especifique si tiene algún equipamiento especial disponible como pueder ser horno, sartén, olla de presión. [Opcional]: ")
    preferencias_dieteticas = input("Indique cualquier preferencia dietética que tenga (e.g., sin gluten, bajo en carbohidratos, vegano) [Opcional]: ")

    #GENERACIÓN DEL PROMPT
    prompt = (f"Basado en las siguientes especificaciones, escribe una receta de cocina {tipo_cocina} "
              f"que se pueda preparar en {tiempo_preparacion} minutos para {num_porciones} personas. "
              f"La receta debe ser de nivel {nivel_dificultad}.")

    #CAMPOS ADICIONALES
    if ingredientes_principales:
        prompt += f" Debe incluir {ingredientes_principales} como ingredientes principales."
    if equipamiento_disponible:
        prompt += f" Se dispone de {equipamiento_disponible}."
    if preferencias_dieteticas:
        prompt += f" La receta debe ser {preferencias_dieteticas}."

    #AÑADO ESTE ÚLTIMO PROMPT SIGUIENDO LA TÉNCICA DE IN-CONTEXT-INSTRUCTION
    prompt += " Por favor, incluye una lista de ingredientes completa, instrucciones paso a paso y cualquier consejo adicional para mejorar el plato."

    return prompt



Implementada la función para generar el prompt, desarrollo la función para obtener la receta. En este caso, voy a utilizar la api_key proporcionada durante las clases y el modelo gpt-3.5-turbo. Como mensaje voy a generar dos prompts:uno a nivel de sistema que va a utilizar la técnica de RolePLay para que especificarle que se comporte como un chef experto  y otro que va a ser el propio prompt obtenido en la función anterior. Este segundo prompt se va a introducir a nivel de usuario. Por último la función generará como respuesta, el mensaje generado por OpenAI.

In [ ]:
def obtener_receta(prompt):
  client = OpenAI (api_key='sk-proj-jRwv8xxafg5nXJ00E13YT3BlbkFJtnk6RJhN4dpoh3sikN6H')
  response = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [
        {"role": "system", "content": "Eres un chef experto."},
        {"role": "user", "content": prompt}
        ],
    temperature = 0.5)

  return response.choices[0].message.content


Por último, llamo a las funciones y genero la receta.

In [ ]:
if __name__ == "__main__":
    prompt = generar_prompt()
    receta = obtener_receta(prompt)
    print("\nReceta generada:\n")
    print(receta)

Ingrese el tipo de cocina que desea cocinar: Italiana
Ingrese el tiempo de preparación (en minutos): 60
Ingrese el número de porciones: 6
Ingrese el nivel de dificultad: fácil, intermedio o avanzado): Avanzado
¿Cuáles son los ingredientes principales que desea usar? [Opcional]: Arroz
Especifique si tiene algún equipamiento especial disponible como pueder ser horno, sartén, olla de presión. [Opcional]: Horno de leña
Indique cualquier preferencia dietética que tenga (e.g., sin gluten, bajo en carbohidratos, vegano) [Opcional]: 

Receta generada:

Receta de Risotto al horno con champiñones y parmesano

Ingredientes:
- 2 tazas de arroz Arborio
- 1 litro de caldo de pollo
- 1 cebolla grande, picada finamente
- 3 dientes de ajo, picados
- 500g de champiñones, en rodajas
- 1 taza de vino blanco seco
- 1 taza de queso parmesano rallado
- 1/2 taza de mantequilla
- Aceite de oliva
- Sal y pimienta al gusto
- Perejil fresco picado para decorar

Instrucciones:

1. Precalienta el horno de leña a 18

El primer ejemplo que he utilizado es el de la cocina italiana y como puede apreciarse se ha obtenido una receta para preprara risotto en base a los argumentos introducidos en el prompt. Es posible ver como la receta obtenida se adecua a estos parámetros ya que por ejemplo está teniendo en cuenta el horno de leña, el arroz, el tipo de cocina italiana...

Voy a generar otro ejemplo para ver si funciona correctamente con otros parámetros.

In [ ]:
if __name__ == "__main__":
    prompt = generar_prompt()
    receta = obtener_receta(prompt)
    print("\nReceta generada:\n")
    print(receta)

Ingrese el tipo de cocina que desea cocinar: Española
Ingrese el tiempo de preparación (en minutos): 120
Ingrese el número de porciones: 4
Ingrese el nivel de dificultad: fácil, intermedio o avanzado): Intermedio
¿Cuáles son los ingredientes principales que desea usar? [Opcional]: Bacalao
Especifique si tiene algún equipamiento especial disponible como pueder ser horno, sartén, olla de presión. [Opcional]: 
Indique cualquier preferencia dietética que tenga (e.g., sin gluten, bajo en carbohidratos, vegano) [Opcional]: Sin gluten

Receta generada:

Receta de Bacalao al Pil Pil

Ingredientes:
- 800g de lomos de bacalao desalado
- 1 taza de aceite de oliva virgen extra
- 4 dientes de ajo, en rodajas finas
- 1 guindilla roja, sin semillas y picada
- Sal y pimienta al gusto
- Perejil fresco picado para decorar

Instrucciones:

1. Seca bien los lomos de bacalao con papel de cocina y córtalos en trozos medianos. Reserva.

2. En una sartén grande a fuego medio, calienta el aceite de oliva. Añad

Puede observarse como funciona correctamente lo implementado ya que he cambiado los parámetros y la receta que ha generado tiene sentido puesto que esta teniendo en cuenta los inputs del usuario. Por ello, doy por concluido este primer apartado.

#Ejercicio 2: Sistema de Resúmenes y Puntos Clave con LangChain o OpenAI

El enunciado de este segundo ejercicio es el siguiente:  En este ejercicio, deberás implementar un sistema que, dado un texto, genere un resumen y liste los puntos clave utilizando LangChain o OpenAI. Este ejercicio tiene como objetivo practicar y asimilar la programación vista durante las clases.

En este segundo apartado se solicita que en base a un texto, el modelo sea capaz de generar un resumen y de extraer los puntos más importantes del texto, Para ello, es posible utilizar tanto OpenAI como LangChain. En mi caso, voy a decantarme por utilizar LangChain para generar el resumen porque me parece que ofrece más posibilidades que OpenAI ya que permite acceder al repositorio de HuggingFace donde existen infinidad de modelos y también me permite implementar una metodología distinta a la implementada en el primer apartado.

Por otro lado, por también trabajar la otra metodología, los puntos claves del resumen los voy a generar con OpenAI de forma similar a como lo he hecho en el ejercicio 1.

El proceso que voy a seguir para resolver este ejercicio va a ser el siguiente:

* En primer lugar, voy a generar una función que se encargue de importar el modelo de HuggingFace.
* En segundo lugar, voy a generar el resumen del texto en base al modelo especificado en la primera función.
* Por último, generaré los puntos claves del resumen por medio de OpenAI.

Creo esta metodología puede ser muy interesante ya que si se implementa correctamente va a permitir automatizar todo el proceso de forma sencilla. Es por eso que definido el proceso, instalo en primer lugar las librerías necesarias.


In [ ]:
pip install langchain langchain-openai langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.3/122.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


En primer lugar genero una función para cargar el modelo de HuggingFaceHub. Esto creo que puede ser interesante para que se pueda probar a generar resúmenes con otros modelos. Simplemente cambiando el ID del modelo, la función devuelve el modelo. Es importante resaltar que el modelo que se desee probar debe haber sido entrenado para resumir textos.

In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

def cargar_modelo(repo_id):
    try:
        modelo = AutoModelForSeq2SeqLM.from_pretrained(repo_id)
        tokenizador = AutoTokenizer.from_pretrained(repo_id)
        return modelo, tokenizador
    except Exception as e:
        print(f"Error al cargar el modelo: {e}")
        return None, None

Seguidamente voy a implementar una función para generar el resumen. Al igual que antes, creo que es interesante implementarlo con una función porque así de este modo se pueden generar resumenes de forma más sencilla y automática. Para ello, en el primer paso de la función se carga el modelo y su tokenizador asociado utilizando la función definida en el apartado anterior. Cargado el tokenizador, se procesa el texto y una vez procesado, se utiliza el modelo para generar el resumen.

In [15]:
def generar_resumen(texto, repo_id):
    # Cargar el modelo y el tokenizador
    modelo, tokenizador = cargar_modelo(repo_id)
    if modelo is None or tokenizador is None:
        print("No se pudo cargar el modelo.")
        return None

    # Preprocesar el texto
    texto_preprocesado = tokenizador(texto, return_tensors="pt", max_length=1024, truncation=True, padding=True)

    # Generar el resumen
    resumen_ids = modelo.generate(input_ids=texto_preprocesado["input_ids"],
                                  attention_mask=texto_preprocesado["attention_mask"],
                                  max_length=1024, num_beams=4, early_stopping=True)

    # Decodificar el resumen
    resumen_texto = tokenizador.decode(resumen_ids[0], skip_special_tokens=True)

    return resumen_texto


Una vez que se genera el texto, el enunciado solicita que se genere los puntos clave del mismo. El resumen lo voy a generar utilizando modelos de HuggingFace pero para generar los puntos claves lo voy a hacer por medio de OpenAI como en el apartado anterior. La función va a ser parecida a la del ejercicio 1 pero teniendo en cuenta que ahora la entrada como prompt es el resumen generado por la función anterior.

In [11]:
from openai import OpenAI

def obtener_puntos_clave(resumen, topico):

    # Inicializar el cliente de OpenAI con tu API key
    client = OpenAI (api_key='sk-proj-jRwv8xxafg5nXJ00E13YT3BlbkFJtnk6RJhN4dpoh3sikN6H')

    prompt = (f"Based on the summary, extract the key points of it. The summary is: {resumen}")

    try:
        # Generar puntos clave utilizando la API de OpenAI
        response = client.chat.completions.create(
            model='gpt-3.5-turbo',
            messages = [
              {"role": "system", "content": f"You are an expert in {topico}"},
              {"role": "user", "content": prompt}
              ],
    temperature = 0.5)

        # Devolver los puntos clave generados por la API
        return response.choices[0].message.content

    except Exception as e:
        print(f"Error al obtener puntos clave: {e}")
        return None




Implementadas las funciones, procedo a realizar una prueba. Como puede apreciarse en la función anterior, he especificado a OpenAI que se comporte como si fuera un experto en función del texto que se introduzca porque a la hora de resumir textos es importante tener en cuenta el tópico del mismo ya que en base a ello se debe utilizar un modelo u otro. Esto es así porque los modelos se entrenan con datos y se ajustan en base a esos datos. Si se utiliza un tipo de dato con el que modelo no ha sido entrenado, lo normal es que no ofrezca el mismo rendimiento que si se utilizará con un tipo de dato con el que ha sido entrenado. Es decir, por ejemplo, si un modelo ha sido entrenado para generar texto en inglés sobre el tópico de viajes, si se le introduce un texto en español que trata de geografía, lo normal será que su rendimiento no sea tan bueno. Es por eso por lo que es importante conocer el propósito del modelo a utilizar y sus datos de entrenamiento.

Para la primera prueba, voy a intentar resumir noticias escritas en inglés ya que considero que en las noticias las ideas clave suelen ser una o dos y lo demás puede ser omitido. Es por eso que necesito encontrar un modelo que haya sido entrenado con noticias en inglés. Investigando la página de HuggingFace es posible apreciar como existen varios modelos que podrían adaptarse pero tras realizar una comparación voy a decanterme por el modelo t5-small. Este modelo ha sido desarrollado por google por lo que ha sido ampliamente utilizado y probado y además de su buen desempeño en textos cortos y medianos, destaca por su eficiencia computacional. Es por eso que en la primera prueba voy a utilizar este modelo.

El texto que he elegido para la prueba lo he extraido del periódico* The Times* y he intentado elegir una noticia neutra. Para seguir teniendo el código organizado, he creado una función para que el usuario ingrese el texto que desea resumir.

In [8]:
def solicitar_texto():
    # Solicitar al usuario que ingrese el texto
    texto = input("Por favor, ingrese el texto que desea resumir: ")

    return texto

In [12]:
#Ingresar el texto
texto = solicitar_texto()

# Especificar el ID del modelo
repo_id = "t5-small"

# Llamar a la función para generar el resumen
resumen = generar_resumen(texto, repo_id)


# Tópico: Es importante especificarlo para que se le pueda indicar a OpenAI como debe comportarse
topico = "politics"

# Llamo a la función para obtener los puntos clave
puntos_clave = obtener_puntos_clave(resumen, topico)

# Imprimo los puntos clave por pantalla
print("------------------------------------------")
print("Texto a resumir:")
print(texto)
print()

# Imprimir el resumen por pantalla
print("Resumen del texto:")
print(resumen)
print()

print("Puntos clave del resumen:")
print(puntos_clave)

Por favor, ingrese el texto que desea resumir: Donald Trump has promised that his relationship with President Putin will secure the release of Evan Gershkovich, the Wall Street Journal reporter being held in Moscow on espionage charges, if he wins the American presidential election in November.  Gershkovich, 32, was arrested in Russia’s Urals region in March last year on suspicion of seeking to obtain defence secrets for United States intelligence. He faces up to 20 years in a penal camp if convicted. Gershkovich and the WSJ have denied the allegation, while the White House has classified him as “wrongfully detained”.  “Evan Gershkovich, the Reporter from The Wall Street Journal, who is being held by Russia, will be released almost immediately after the Election, but definitely before I assume Office. He will be HOME, SAFE, AND WITH HIS FAMILY,” Trump wrote on his Truth Social website.  Donald Trump made the claims on Truth Social but the Kremlin says President Putin has had no contact

Como puede verse tanto el resumen como los puntos claves funcionan perfectamente ya que ambos dos cumplen con su cometido. El resumen ofrece brevemente las ideas principales de la noticia y luego gracias a OpenAI se genera un listado con los puntos claves. Esto sin duda es muy interesante porque especificando unicamente el modelo, el topico de la noticia e indicando la propia noticia, se puede extraer su información más relevante.

Por probar otro modelo, voy a realizar el mismo ejercicio pero en este caso resumiendo una oferta de empleo en inglés. En este caso he investigado HuggingFace y voy a utilizar el modelo facebook/bart-base porque la arquitectura de los modelos BART es perfecta para generar textos y porque posee una gran capacidad para capturar patrones complejos.

In [16]:
#Ingresar el texto
texto = solicitar_texto()

# Especificar el ID del modelo
repo_id = "facebook/bart-large-cnn"

# Llamar a la función para generar el resumen
resumen = generar_resumen(texto, repo_id)

# Tópico: Es importante especificarlo para que se le pueda indicar a OpenAI como debe comportarse
topico = "hiring"

# Llamo a la función para obtener los puntos clave
puntos_clave = obtener_puntos_clave(resumen, topico)

# Imprimo los puntos clave por pantalla
print("------------------------------------------")
print("Texto a resumir:")
print(texto)
print()

# Imprimir el resumen por pantalla
print("Resumen del texto:")
print(resumen)
print()

print("Puntos clave del resumen:")
print(puntos_clave)

Por favor, ingrese el texto que desea resumir: We are hiring a Junior AI Engineer to support the engineering team with multiple aspects of agent development, including system design, prompt engineering, and integrations.   You will play a crucial role in designing and implementing agentic workflows, developing & refining AI prompts, integrating agents with various data sources and software tools, and evaluating model/agent performance. This role offers a unique opportunity to gain hands-on experience in developing AI systems for real-world applications. The ideal candidate will have some experience with scripting languages (Python & TypeScript required), a deep personal interest in AI, and a strong sense of system design.   Responsibilities  Agentic Workflow Design: Collaborate with the AI team to conceptualize and build structured workflows for AI agents, defining clear objectives and tasks. Prompt Engineering: Craft effective prompts to guide agent behaviors and interactions, optimiz

En este caso también es posible apreciar como con este nuevo modelo es posible generar un resumen y los puntos claves del mismo. De lo que parecía mucha información sobre lo que solicitaba la empresa, se ha obtenido un listado y un resumen que recoge la esencia la oferta de empleo. Esto es muy intersante también porque se podría implementar algún automatismo que procesará ofertas de empleo de portales de empleo y para cada una, generará el resumen y los puntos más importantes. Esto por ejemplo sería interesante para empresas que se dedican a la contratación ya que podría tener una visión global de lo que se busca en el mercado.

Por último, destacar que en este caso los dos ejemplos que se han elegido han sido basados en lo que en mi opinión personal era interesante resumir. Esto no quiere decir que la solución no pueda ser utilizada con otros textos. Asimismo, las dos soluciones que se han generado han sido en inglés porque globalmente el idioma oficial de programación y por lo tanto de HuggingFace es en inglés y como consecuencia hay muchos más recursos en este idioma. Es por eso que las dos soluciones son en este idioms. No obstante, también se podría utilizar la solución con modelos en español, ya que lo único que habría que cambiar sería el ID del modelo. Esto es la ventaja de haberlo implementado con funciones.

#Agradecimientos

Para finalizar esta práctica deseo agradecer a nuestro profesor Marc Mayol por todas las horas que ha invertido en preparar las clases, impartirlas, transmitir todos los conocimientos, resolver todas las dudas y estar pendiente de que entendamos los conceptos claves del prompt engineering. Aunque este módulo haya sido corto creo que nos ha permitido conocer los conceptos básicos del prompt engineering y forma en que se puede automatizar por medio de scripts. Desde mi punto de vista, esto es muy interesante dado que puede permitir generar soluciones que pueden ser muy valiosas a la hora de tomar decisiones Además, en mi opinión creo que fue un acierto que Marc dedicará mucho tiempo a explicar HuggingFace ya que el hecho de conocer este repositorio permite generar mejores soluciones gracias a los modelos ya preentrenados. Es por eso por lo que quiero expresar mi más sincero agradecimiento a nuestro profesor.

Por último, agradezco también a la academia Keepcoding por haber introducido este módulo en el bootcamp, el cual creo que va a ser muy beneficioso para mi futuro.